In [ ]:
!pip install fastapi uvicorn pydantic requests

In [ ]:
!npm install -g localtunnel

In [ ]:
import os
import time

OLLAMA_MODEL = "llama3:8b-instruct-q4_0"
print("1. Installing Ollama...")
os.system('curl -fsSL https://ollama.com/install.sh | sh')

print("2. Starting Ollama daemon in background...")
os.system('nohup ollama serve &') 

time.sleep(5)
print(f"3. Pulling model: {OLLAMA_MODEL}...")
os.system(f'ollama pull {OLLAMA_MODEL}')

time.sleep(5)

print("Ollama setup complete. Service should be running on http://localhost:11434.")

In [ ]:
%%writefile main.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import requests
import json
from datetime import date

# Khai báo Ứng dụng và Cấu hình

app = FastAPI(title="AI Travel Itinerary Backend")

OLLAMA_ENDPOINT = "http://localhost:11434/api/generate"
OLLAMA_MODEL = "llama3:8b-instruct-q4_0" 

# Khai báo Mô hình Dữ liệu (Pydantic)

class ItineraryRequest(BaseModel):
    origin: str
    destination: str
    start_date: date
    end_date: date
    interests: List[str]
    pace: str

class ItineraryResponse(BaseModel):
    status: str
    message: str
    itinerary: dict = None

# Hàm gọi LLM và Xử lý Prompt

def generate_llm_prompt(data: ItineraryRequest) -> str:
    
    num_days = (data.end_date - data.start_date).days + 1
    interests_str = ", ".join(data.interests)
    
    prompt = f"""
    Bạn là một trợ lý du lịch AI. Nhiệm vụ của bạn là tạo ra một hành trình chi tiết.
    
    Yêu cầu chuyến đi:
    - Điểm đi: {data.origin}
    - Điểm đến: {data.destination}
    - Ngày: Từ {data.start_date} đến {data.end_date} (Tổng cộng {num_days} ngày)
    - Sở thích: {interests_str}
    - Tốc độ: {data.pace}
    
    Định dạng đầu ra:
    Bạn phải trả lời bằng định dạng JSON hợp lệ, KHÔNG CHỨA bất kỳ ký tự hoặc giải thích nào nào khác ngoài JSON.
    Cấu trúc JSON PHẢI tuân theo schema sau:
    {{
        "itinerary": [
            {{
                "day": 1,
                "date": "YYYY-MM-DD",
                "activities": [
                    {{
                        "time": "Sáng",
                        "place": "Tên địa điểm",
                        "explanation": "Giải thích ngắn gọn (Vd: Tham quan, ăn uống)"
                    }},
                    // ... các hoạt động khác: "Trưa", "Chiều", "Tối"
                ]
            }},
            // ... các ngày khác
        ]
    }}
    Bắt đầu tạo hành trình ngay bây giờ:
    """
    return prompt

# Endpoint API

@app.post("/generate_itinerary", response_model=ItineraryResponse)
async def generate_itinerary(data: ItineraryRequest):
    """Xử lý yêu cầu từ Frontend và gọi LLM thông qua Ollama."""
    
    full_prompt = generate_llm_prompt(data)
    
    ollama_payload = {
        "model": OLLAMA_MODEL,
        "prompt": full_prompt,
        "format": "json", 
        "stream": False
    }

    try:
        response = requests.post(
            OLLAMA_ENDPOINT, 
            json=ollama_payload, 
            timeout=300
        )
        response.raise_for_status()
        
        ollama_response = response.json()
        raw_output = ollama_response.get("response", "{}")
        
        try:
            itinerary_json = json.loads(raw_output)
            
            return ItineraryResponse(
                status="success",
                message="Hành trình được tạo thành công.",
                itinerary=itinerary_json
            )
        except json.JSONDecodeError as e:
            return ItineraryResponse(
                status="error",
                message=f"LLM trả về JSON không hợp lệ. Lỗi: {e}",
                itinerary={"raw_output": raw_output}
            )

    except requests.exceptions.RequestException as e:
        raise HTTPException(
            status_code=503, 
            detail=f"Lỗi kết nối đến Ollama Backend: {e}. Vui lòng đảm bảo Ollama đang chạy trên cổng 11434."
        )

In [ ]:
# Dọn dẹp cổng
!fuser -k 8005/tcp

import subprocess
import time
import os
import signal

FASTAPI_PORT = 8005

print(f"Starting FastAPI server on port {FASTAPI_PORT} using subprocess...")

try:
    uvicorn_process = subprocess.Popen(
        ['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', str(FASTAPI_PORT)],
        preexec_fn=os.setsid 
    )
    print(f"FastAPI is running (PID: {uvicorn_process.pid}).")
except Exception as e:
    print(f"LỖI KHỞI ĐỘNG UVICORN: {e}")
    raise

time.sleep(5)

print("Starting LocalTunnel...")
print("Vui lòng truy cập URL hiển thị dưới đây:")
get_ipython().system(f'lt --port {FASTAPI_PORT}')